## 5.다항 분류
- Multi Classification
- 3 개 이상의 Label을 갖는 데이터에 대한 분류

### 5-01. 함수 생성

3과목의 평균에 따라 A, B, C, D, F 학점으로 분류
- 90 이상 : 'A'
- 80 이상 : 'B'
- 70 이상 : 'C'
- 60 이상 : 'D'
- 60 미만 : 'F'

In [28]:
# [0] 사용할 라이브러리 import
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [29]:
# [1] 원하는 수의 행(sample)을 갖는 데이터 작성
# seedno : 랜덤 수 생성 규칙
# size : 랜덤 수 생성 행의 수
# step=0 (비교적 ^^ 균형), step=다른수 (불균형)

def make_sample(seedno,size,step=0):
  np.random.seed(seedno)
  if step == 0:
    size = int(size/4)
    A = pd.DataFrame(np.random.randint(0,101,(size,3)))
    B = pd.DataFrame(np.random.randint(85,101,(size,3)))
    C = pd.DataFrame(np.random.randint(70,95,(size,3)))
    D = pd.DataFrame(np.random.randint(60,80,(size,3)))
    data = pd.concat([A,B,C,D],ignore_index=True)
  else:
    data = pd.DataFrame(np.random.randint(0,101,(size,3)))

  data.columns = ['국어','영어','수학']
  s = data.mean(axis=1)
  data['학점'] = s.apply(lambda x:'A' if x>=90 else 'B' if x>=80 else 'C' if x>=70 else 'D' if x>=60 else 'F')

  print(data['학점'].value_counts().sort_index())
  data['학점'] = data['학점'].replace({'A':0,'B':1,'C':2,'D':3,'F':4})
  return data

In [30]:
# [2] 모든 경우의 수로 데이터 작성 : 101 * 101 * 101 = 1,030,301
def make_all():
  colnames = ['국어','영어','수학']
  data = [[kor,eng,math] for kor in range(101)
                         for eng in range(101)
                         for math in range(101)]
  data = pd.DataFrame(data,columns = colnames)
  s = data.mean(axis=1)
  data['학점'] = s.apply(lambda x: 'A' if x>= 90 else 'B' if x>=80 else 'C' if x>=70 else 'D' if x>= 60 else 'F')
  print(data['학점'].value_counts().sort_index())
  data['학점'] = data['학점'].replace({'A':0,'B':1,'C':2,'D':3,'F':4})
  return data

In [31]:
# [3] 모델 학습 및 성능 평가 (train 데이터 사용)
from sklearn.preprocessing import StandardScaler
def ModelTrain(model,df):
  #X,Y 데이터 준비
  Y = df['학점']
  X = df.drop(columns=['학점'])
#

  #train,test 데이터 분할
  xtrain,xtest,ytrain,ytest = train_test_split(X,Y,
                                               stratify = Y, random_state=0,test_size=0.3)
  #모델 선택 및 학습
  model.fit(xtrain,ytrain)

  #성능 평가 -accuracy,roc_auc_score
  A = model.score(xtrain,ytrain)
  B = model.score(xtest,ytest)
  ypred = model.predict_proba(xtest)
  C = roc_auc_score(ytest,ypred,multi_class='ovo',average='weighted')
  print(f'{A:.4f} {B:.4f} {C:.4f}')
  return model

In [32]:
# multi_class='raise' : 멀티클래스일 때 오류 발생
# multi_class='ovo' : One-vs-one을 의미
# 가능한 모든 클래스 쌍 조합의 평균 AUC를 계산
# multi_class='ovr' : One-vs-rest를 의미
# 나머지에 대해 각 클래스의 AUC를 계산
# 클래스 불균형은 각 '나머지'그룹의 구성에 영향을 미치기 때문에
# average == 'macro' 일 때도 클래스 불균형에 민감

# average='macro' : 레이블 불균형은 고려되지 않음
# average='weighted' : 레이블에 가중치가 부여된 평균
# https://runebook.dev/ko/docs/scikit_learn/modules/generated/sklearn.metrics.roc_auc_score

In [33]:
# [4] 결과 분석용 함수
# data : 2D [[국어, 영어, 수학], ...]
def get_result(data,model):
  df = pd.DataFrame(data,columns=['국어','영어','수학'])
  df['평균'] = df.mean(axis=1)
  df['학점'] = df['평균'].apply(lambda x : 'A' if x>=90 else 'B' if x>=80 else 'C' if x>= 70 else 'D' if x>=60 else 'F')
  df['예측'] = model.predict(df[['국어','영어','수학']])
  df['예측'] = df['예측'].replace({0:'A',1:'B',2:'C',3:'D',4:'F'})
  return df

### 5-02.데이터 생성 및 분류

In [34]:
# [5] 사용할 모델 import
from sklearn.linear_model import LogisticRegression

In [35]:
# [6] LogisticRegression 사용 (max_iter=1000)
# 데이터 : seedno=1234, size=30000
data = make_sample(seedno=1234,size=30000)
#display(data.head(2))
model_A = LogisticRegression(max_iter=1000)
ModelTrain(model_A,data)

학점
A    6483
B    6419
C    6531
D    5247
F    5320
Name: count, dtype: int64
1.0000 1.0000 1.0000


LogisticRegression(max_iter=1000)

In [36]:
# [7] 결과 확인 - 경계값 중심
data = [[9, 100, 100], [89, 92, 88], [100, 98, 72],
        [60, 60, 60], [60, 60, 59], [59, 60, 60],
        [90, 98, 82], [100, 100, 10], [100, 100, 9]]
get_result(data,model_A)

,국어,영어,수학,평균,학점,예측
0,9,100,100,69.666667,D,D
1,89,92,88,89.666667,B,B
2,100,98,72,90.000000,A,A
3,60,60,60,60.000000,D,D
4,60,60,59,59.666667,F,F
5,59,60,60,59.666667,F,F
6,90,98,82,90.000000,A,A
7,100,100,10,70.000000,C,C
8,100,100,9,69.666667,D,D


In [37]:
# [8] 데이터 개수 줄이고, 불균형으로 만듦
# LogisticRegression 사용 (max_iter=8000)
# seedno=1234, size=5000, step=1
data = make_sample(seedno=1234,size=5000,step=1)
display(data.head())
model_B = LogisticRegression(max_iter=8000)
ModelTrain(model_B,data)

학점
A      21
B     172
C     453
D     815
F    3539
Name: count, dtype: int64


,국어,영어,수학,학점
0,47,83,38,4
1,53,76,24,4
2,15,49,23,4
3,26,30,43,4
4,30,26,58,4


1.0000 0.9993 1.0000


LogisticRegression(max_iter=8000)

In [38]:
# [9] 결과 확인 - 경계값 중심
data = [[9, 100, 100], [89, 92, 88], [100, 98, 72],
        [60, 60, 60], [60, 60, 59], [59, 60, 60],
        [90, 98, 82], [100, 100, 10], [100, 100, 9]]
get_result(data,model_B)

,국어,영어,수학,평균,학점,예측
0,9,100,100,69.666667,D,C
1,89,92,88,89.666667,B,B
2,100,98,72,90.000000,A,B
3,60,60,60,60.000000,D,D
4,60,60,59,59.666667,F,F
5,59,60,60,59.666667,F,F
6,90,98,82,90.000000,A,A
7,100,100,10,70.000000,C,C
8,100,100,9,69.666667,D,D


In [39]:
# [10] dataAll 생성 (모든 경우의 수를 생성함)
dataAll = make_all()
dataAll.shape

학점
A      5456
B     34255
C     90055
D    168235
F    732300
Name: count, dtype: int64


(1030301, 4)

In [40]:
# [11] dataAll을 사용해 modelA, modelB를 평가
data = dataAll[['국어','영어','수학']]
resultA = get_result(data,model_A)
print('model_A:'  ,sum(resultA['학점'] !=resultA['예측'] ))
resultB = get_result(data,model_B)
print('model_B:' , sum(resultB['학점'] != resultB['예측']))

model_A: 0
model_B: 1169


In [41]:
# [12] 틀린 것의 목록 만들기
wrong = resultB[resultB['학점'] != resultB['예측']]
display(wrong.tail())

,국어,영어,수학,평균,학점,예측
1030070,100,98,72,90.0,A,B
1030140,100,99,41,80.0,B,C
1030170,100,99,71,90.0,A,B
1030240,100,100,40,80.0,B,C
1030270,100,100,70,90.0,A,B


In [42]:
# [13] 틀린 것의 피벗테이블 작성 (오분류표)
resultB.pivot_table(index='학점',columns = '예측',
                    values ='국어',aggfunc='count',fill_value=0)

예측,A,B,C,D,F
학점,,,,,
A,5390,66,0,0,0
B,308,33508,439,0,0
C,0,352,89703,0,0
D,0,0,4,168231,0
F,0,0,0,0,732300


### 5-03.오분류표

sklearn.metrics.confusion_matrix(y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)

In [43]:
# [14] 오분류표 작성
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(resultB['학점'],resultB['예측'])
feature_names =list("ABCDF")
df_cm = pd.DataFrame(cm,columns = feature_names,index=feature_names)
df_cm.index.name = '실제'
df_cm.columns.name = '예측'
df_cm

예측,A,B,C,D,F
실제,,,,,
A,5390,66,0,0,0
B,308,33508,439,0,0
C,0,352,89703,0,0
D,0,0,4,168231,0
F,0,0,0,0,732300


### 5-04.여러 모델 사용
- 어떤 모델을 선택해야 할까?
- accuracy(train, test), roc_auc_score(test) 모두 좋은 것!
- train, test가 비슷한 것!

In [44]:
# 모델 import
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [45]:
# [15] seedno=1234, size=30000 의 데이터 작성
data = make_sample(seedno=1234,size=30000)
data.shape

학점
A    6483
B    6419
C    6531
D    5247
F    5320
Name: count, dtype: int64


(30000, 4)

In [46]:
# [16] KNeighborsClassifier 사용 (k=1~9)
dataX = dataAll[['국어','영어','수학']]
for k in range(1,10):
  model_C = KNeighborsClassifier(k)
  ModelTrain(model_C,data)
  result = get_result(dataX,model_C)
  print('wrong cnt :',sum(result['학점'] != result['예측']))

1.0000 0.9807 0.9878
wrong cnt : 36895
0.9913 0.9752 0.9950
wrong cnt : 43512
0.9912 0.9794 0.9975
wrong cnt : 35221
0.9869 0.9770 0.9989
wrong cnt : 38672
0.9860 0.9788 0.9993
wrong cnt : 34976
0.9850 0.9796 0.9993
wrong cnt : 36728
0.9852 0.9801 0.9993
wrong cnt : 34746
0.9844 0.9804 0.9993
wrong cnt : 35830
0.9847 0.9812 0.9994
wrong cnt : 35006


In [47]:
# [17] DecisionTreeClassifier 사용 (d=3~11)
model_D = DecisionTreeClassifier(random_state=1234)
ModelTrain(model_D,data)
result = get_result(dataX,model_D)
print('wrong cnt : ' , sum(result['학점'] != result['예측']))

for d in range(3,12):
  model_D = DecisionTreeClassifier(max_depth=d,random_state=0)
  ModelTrain(model_D,data)
  result = get_result(dataX,model_D)
  print('wrong cnt : ', sum(result['학점'] != result['예측']))

1.0000 0.9678 0.9797
wrong cnt :  63441
0.6605 0.6596 0.8901
wrong cnt :  393632
0.7607 0.7583 0.9335
wrong cnt :  345380
0.8181 0.8077 0.9595
wrong cnt :  210188
0.8714 0.8586 0.9741
wrong cnt :  168396
0.9120 0.8897 0.9808
wrong cnt :  131759
0.9313 0.9091 0.9845
wrong cnt :  101026
0.9496 0.9253 0.9852
wrong cnt :  85611
0.9676 0.9418 0.9833
wrong cnt :  72726
0.9804 0.9534 0.9831
wrong cnt :  70244


In [48]:
# [18] RandomForestClassifier
model_E = RandomForestClassifier(random_state=1234)
ModelTrain(model_E,data)
result = get_result(dataX,model_E)
print('wrong cnt :', sum(result['학점'] != result['예측']))

for h in range(3,12):
  model_E = RandomForestClassifier(500,max_depth=h,random_state=1234)
  ModelTrain(model_E,data)
  result = get_result(dataX,model_E)
  print('wrong cnt :', sum(result['학점'] != result['예측']))


1.0000 0.9792 0.9995
wrong cnt : 46145
0.8296 0.8268 0.9722
wrong cnt : 255694
0.8523 0.8497 0.9801
wrong cnt : 234630
0.8840 0.8764 0.9854
wrong cnt : 197121
0.9188 0.9059 0.9908
wrong cnt : 137557
0.9460 0.9337 0.9948
wrong cnt : 91772
0.9655 0.9451 0.9969
wrong cnt : 68358
0.9788 0.9609 0.9981
wrong cnt : 55885
0.9889 0.9694 0.9987
wrong cnt : 50338
0.9944 0.9738 0.9991
wrong cnt : 46883
